In [ ]:
# 모델 파이프라인으로 불러오기

import torch
import transformers
MY_HF_TOKEN = "input your key in huggingface"

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token = MY_HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
# 학습시킬 데이터 csv 파일 불러와서 각 질병마다 하나의 텍스트로 정리

import pandas as pd

# disease_path = '/content/drive/MyDrive/제4회 AI교육 해커톤/disease_train.csv'
disease_path = 'input disease path'
data = pd.read_csv(disease_path)

data['text'] = data.apply(
    lambda row: f"질병: {row['질병']}\n설명: {row['설명']}\n원인: {row['원인']}\n증상: {row['증상']}\n치료방법: {row['치료방법']}",
    axis=1
)

원래는 data['text']를 불러온 모델에 학습시킨 후 예측된 질병에 따라 설명을 생성해야 하는데,

RAM 한계로 프롬프트에 각 질병의 정보를 입력하여 예측된 질병에 따라 설명하는 결과를 출력하도록 변경

프롬프트가 아닌 모델에 학습시키는 코드는 'trained_llama.ipynb' 참고

In [ ]:
# 예측된 질병이 '성충_날개불구바이러스감염증'일 때 설명 결과 출력

CLASS = ['성충_날개불구바이러스감염증',
        '성충_응애',
        '성충_정상',
        '유충_부저병',
        '유충_석고병',
        '유충_응애',
        '유충_정상']

messages = [
    {"role": "system", "content": "You are good at describing seven diseases of honeybees."},
    {"role": "user", "content": "'성충_날개불구바이러스감염증' 질병에 대해 한글로 4문장으로 설명해줘."},
]

# 각 질병에 대해 모델을 호출
for idx, row in data.iterrows():
    messages[0]["content"] = row['text']

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

generated_text = outputs[0]["generated_text"][-1]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': 'Here are four sentences explaining the disease "성충_날개불구바이러스감염증" in Korean:\n\n날개불구바이러스감염증은 꿀벌의 체액을 빨아먹는 바로아 응애에 의해 전파되는 바이러스성 질병입니다. 이 질병으로 감염된 꿀벌은 날개 기형을 일으키고 비행 능력을 상실하게 됩니다. 군체 내에서 바이러스가 전염되면 꿀벌의 활력이 떨어지고 벌 개체 수가 급격히 감소하는 등 치명적인 결과를 초래할 수 있습니다. 이를 예방하고 치료하려면 응애를 제거하고 화학적 방법으로 항바이러스 치료를 수행하며 군체 관리를 철저히 해야 합니다.'}


In [ ]:
# 예측된 질병이 '성충_응애'일 때 설명 결과 출력

CLASS = ['성충_날개불구바이러스감염증',
        '성충_응애',
        '성충_정상',
        '유충_부저병',
        '유충_석고병',
        '유충_응애',
        '유충_정상']

messages = [
    {"role": "system", "content": "You are good at describing seven diseases of honeybees."},
    {"role": "user", "content": "'성충_응애' 질병에 대해 한글로 4문장으로 설명해줘."},
]

# 각 질병에 대해 모델을 호출
for idx, row in data.iterrows():
    messages[0]["content"] = row['text']

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

generated_text = outputs[0]["generated_text"][-1]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': 'Here are four sentences explaining the disease "성충_응애" in Korean:\n\n성충_응애는 꿀벌의 날개 기형을 일으키는 바이러스성 질병입니다. 이 질병은 바로아 응애가 꿀벌의 체액을 빨아먹으며 바이러스를 전달하는 과정에서 발생합니다. 감염된 꿀벌은 날개 기형을 일으키고 비행 능력을 상실하여 짧은 수명을 갖게 됩니다. 이를 방지하기 위해 응애 제거, 항바이러스 치료, 군체 관리 등을 통해 질병을 관리해야 합니다.'}


In [ ]:
# 예측된 질병이 '유충_부저병'일 때 설명 결과 출력

CLASS = ['성충_날개불구바이러스감염증',
        '성충_응애',
        '성충_정상',
        '유충_부저병',
        '유충_석고병',
        '유충_응애',
        '유충_정상']

messages = [
    {"role": "system", "content": "You are good at describing seven diseases of honeybees."},
    {"role": "user", "content": "'유충_부저병' 질병에 대해 한글로 4문장으로 설명해줘."},
]

for idx, row in data.iterrows():
    messages[0]["content"] = row['text']

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

generated_text = outputs[0]["generated_text"][-1]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': 'Here are 4 sentences explaining the disease "유충_부저병" in Korean:\n\n유충_부저병은 꿀벌의 유충이 부저에 감염되는 바이러스성 질병입니다. 이 질병은 꿀벌의 유충이 부저에 있는 바이러스에 감염되어 부저가 부식되거나 파괴되는 것을 일으킵니다. 이러한 질병은 꿀벌의 생산을 저해하고, 꿀벌의 생존을 위협합니다. 질병을 예방하고 치료하기 위해서는 꿀벌의 유충을 적절하게 관리하고, 부저를 청결하게 하는 것이 중요합니다.'}


In [ ]:
# 예측된 질병이 '유충_석고병'일 때 설명 결과 출력

CLASS = ['성충_날개불구바이러스감염증',
        '성충_응애',
        '성충_정상',
        '유충_부저병',
        '유충_석고병',
        '유충_응애',
        '유충_정상']

messages = [
    {"role": "system", "content": "You are good at describing seven diseases of honeybees."},
    {"role": "user", "content": "'유충_석고병' 질병에 대해 한글로 4문장으로 설명해줘."},
]

# 각 질병에 대해 모델을 호출
for idx, row in data.iterrows():
    messages[0]["content"] = row['text']

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

generated_text = outputs[0]["generated_text"][-1]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': 'Here is a 4-sentence explanation of the "Yu-chung_Seok-go-byeong" disease in Korean:\n\n유충_석고병은 꿀벌의 유충이 석고균에 감염된 질병입니다. 이 질병은 유충이 석고균에 의해 파괴되는 경우를 초래하고, 이에 따라 꿀벌의 생산이 저하됩니다. 석고균은 꿀벌의 유충을 죽이는 데 사용되는 화학 물질을 생산하는 데 사용됩니다. 이 질병은 꿀벌의 생산을 저하하고, 꿀벌의 건강을 악화시킬 수 있습니다.'}


In [ ]:
# 예측된 질병이 '유충_응애'일 때 설명 결과 출력

CLASS = ['성충_날개불구바이러스감염증',
        '성충_응애',
        '성충_정상',
        '유충_부저병',
        '유충_석고병',
        '유충_응애',
        '유충_정상']

messages = [
    {"role": "system", "content": "You are good at describing seven diseases of honeybees."},
    {"role": "user", "content": "'유충_응애' 질병에 대해 한글로 4문장으로 설명해줘."},
]

# 각 질병에 대해 모델을 호출
for idx, row in data.iterrows():
    messages[0]["content"] = row['text']

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

generated_text = outputs[0]["generated_text"][-1]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': 'Here are 4 sentences explaining the "유충_응애" disease in Korean:\n\n유충_응애는 꿀벌의 유충이 응애에 의해 감염되는 바이러스성 질병입니다. 이 질병은 꿀벌의 유충이 응애에 기생당하여 바이러스를 전달받아 감염됩니다. 감염된 꿀벌의 유충은 체력이 약해지게 되고, 성충이 되지 못하는 경우가 많습니다. 이 질병은 꿀벌 군체의 활력을 떨어뜨리고, 벌 개체 수를 감소시킬 수 있습니다.'}
